
<span style="font-family: 'Times New Roman'; font-size: 24px;">Importing Libraries</span>

In [1]:
import requests
from bs4 import BeautifulSoup
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# Download the CMU Pronouncing Dictionary for syllable counting

from nltk.corpus import cmudict
nltk.download('cmudict')

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import os
import re
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<span style="font-family: 'Times New Roman'; font-size: 24px;">Creating the Dataframe and Webscraping </span>


In [2]:
xlsx = pd.ExcelFile(r'C:\Users\ACER\Downloads\Input (1).xlsx', engine='openpyxl')
df=pd.read_excel(xlsx)

article=[]
for i in df.URL:
    text_content=''
    webpage=requests.get(i)
    soup=BeautifulSoup(webpage.content,'lxml')
    for j in soup.find_all('div',class_=["td-post-content tagdiv-type","tdb-block-inner td-fix-index"]):
        text_content += j.get_text() + '\n'
    article.append(text_content)  

df.insert(2,'article',article,True)

#cleaning the webscraping content
for i in range(0, 100):
    index = df.iloc[i]['article'].find('Search')
    x = str(df.iloc[i]['article'])
    df.iloc[i, df.columns.get_loc('article')] = x[index + 6:]

#saving the articles in a text file:
article_title=[]
for i in df.URL:
    text_content=''
    webpage=requests.get(i)
    soup=BeautifulSoup(webpage.content,'lxml')
    for j in soup.find_all('h1',class_=["entry-title"]):
        text_content += j.get_text() 
    article_title.append(text_content)
df['article_title']=article_title





<span style="font-family: 'Times New Roman'; font-size: 24px;">Text preprocessing</span>


Developing functions for text preprocessing

In [3]:
#lowercasing
df['article']=df['article'].str.lower()

#removing urls
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub('',text)

# removing \n's 
def removingns(text):
    return text.replace('\n',' ')

#removing html 
def remove_html(text):
    html_tags_pattern = re.compile(r'<.*?>')
    return html_tags_pattern.sub(r'',text)

#removing punctuation
def remove_punc(text):
    punctuations=string.punctuation
    for i in punctuations:
        text=text.replace(i,'')
    return text


# extracting of positve and negative stopwords
path=r'C:\Users\ACER\Downloads\StopWords-20240302T184700Z-001\StopWords'
s=[]
stopwords=[]
for i in os.listdir(path):
    file_path=os.path.join(path,i)
    file=open(file_path)
    for i in file:
        s.append(i.strip('\n').lower())
for i in s:
    for k in i.split('|'):
        stopwords.append(k)

# creating master dictionary of positive words and negative words
path=r'C:\Users\ACER\Downloads\MasterDictionary-20240302T110426Z-001\MasterDictionary'
flag=0 #flag for checking postivefile or negativefile
positive_word_list=[]
negative_word_list=[]
for i in os.listdir(path):
    file_path=os.path.join(path,i)
    file=open(file_path)
    for i in file:
        if flag==0:
            if i not in stopwords:
                negative_word_list.append(i.strip('\n'))
        else:
            if i not in stopwords:
                positive_word_list.append(i.strip('\n'))
    flag=flag+1
    
#creating dictionary out of it
positive_word={}
negative_word={}
for word in positive_word_list:
    positive_word[word] = True
for word in negative_word_list:
    negative_word[word]=True


def remove_stopword(text):
    new_text=''
    for word in text.split():
        if word not in stopwords:
            new_text+=word+' '
    return new_text

lemmatizer=WordNetLemmatizer()
def lemmatization(tokens):
    lemmatized_tokens=[lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens


Calling the above defined functions

In [4]:
df['aritcle']=df['article'].apply(remove_url)
df['article']=df['article'].apply(removingns)
df['article']=df['article'].apply(remove_html)

#preserving the article before removing stopwords from it 
df['b4Cleanarticle']=df['article']


df['article']=df['article'].apply(remove_stopword)

#using built-in function for tokenization
df['token']=df['article'].apply(word_tokenize)

df['lemmatized_tokens']=df['token'].apply(lemmatization)


In [5]:
#change the path according to your needs
path=r'E:\text_articles'
i=0
def filetextmaker(row):
    global i
    filename=str(row['URL_ID'])
    file_path=os.path.join(path,filename)
    f=open(file_path+'.txt','w',encoding='utf-8')
    f.write(row['article_title'] +'\n')
    f.write(row['b4Cleanarticle'])
    print(i)
    i+=1
df.apply(filetextmaker,axis=1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object

<span style="font-family: 'Times New Roman'; font-size: 24px;">Making Functions for Output variables (postive,negative,polarity and subjectivity score)</span>


In [6]:
#positivity score
def positivity(lis):
    positive=0
    for i in lis:
        if i in positive_word_list:
            positive=positive+1
    return positive

#negaitivity score
def negativity(lis):
    negative=0
    for i in lis:
        if i in negative_word_list:
            negative=negative-1
    return negative*-1

#subjectivity score
def subjectivity(lis,posit,negat):
    words_afterclean=len(lis)
    subjectivity_score=(posit+negat)/((words_afterclean)+0.000001)
    return subjectivity_score
#polarity score
def polarity(posit,negat):
    polarity_score=(posit-negat)/((posit+negat)+0.000001)
    return polarity_score


Calling the above defined functions

In [7]:
df['positive_score']=df['lemmatized_tokens'].apply(positivity)
df['negative_score']=df['lemmatized_tokens'].apply(negativity)
df['subjectivity_score']=df.apply(lambda row: subjectivity(row['lemmatized_tokens'],row['positive_score'],row['negative_score']),axis=1 )
df['polarity_score'] = df.apply(lambda row: polarity(row['positive_score'], row['negative_score']), axis=1)

Tokenizing

In [8]:
df['sentence_token']=df['b4Cleanarticle'].apply(sent_tokenize)

<p style="font-family: 'Times New Roman'; font-size: 24px;">Making Functions 
 for Output variables 
<p style="font-family: 'Times New Roman'; font-size: 20px;">
avg sentence length<br>
percentage of complex words<br>
fog index<br>
avg number of words per sentence<br>
complex word count<br>
word count<br>
syllable per word<br>
personal pronouns<br>
avg word length
    
</p> </p>


In [9]:
d = cmudict.dict()
def syllable_count(word):
    if word.lower() in d:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]
    else:
        return [1]
def is_complex(word, threshold=3):
    syllables = syllable_count(word)
    return any(s >= threshold for s in syllables)

def sentence_count(lis):
    sent_count=0
    word_key=0
    l=[]
    for i in lis:
        global article_Count
        word_key=word_key+1
        word_value=[]
        word_count=0
        for j in i.split():
            word_count+=1
        word_value.append(word_count)
        sent_count=sent_count+1
        word.append(word_count)
        l=word
    article_Count+=1
    dict[article_Count]=l
    
    if(sent_count!=0):
        sent.append(sent_count)
    else:
        sent.append(sent_count)
        sent_count=1
        
#finding error indexes where division by zero is ocuuring count=0 

def readability(text):
    
    global sent_index,avg_sentencelength,complexpercent,complex_word,fog_index,avg_sentlength
    if(sent_index<100):
        #no. of words in whole text
        word_count=len(text.split())
    #no. of complex words in text
        complex_word = sum(is_complex(word) for word in text.split())
        complex_wordCount.append(complex_word)
    
        avg_sentlength=word_count/sent[sent_index]
      
        avg_sentencelength.append(avg_sentlength)
    
        if(word_count!=0):
            complexpercent=complex_word/word_count
            complex_percent.append(complexpercent)
        else:
            complex_percent.append(complexpercent)
            complexpercent=0
 
    fogindex=0.4*(avg_sentlength+complexpercent)
    fog_index.append(fogindex)
    sent_index+=1
   
    return avg_sentlength,complexpercent,complex_word

syllable_error_index=0

def syllableperword(text):
    global index,syllable_error_index
    #handling exception using regex 
    pattern = r'\b(\w+)(?:es|ed)\b'
    clean_text = re.sub(pattern, r'\1', text)
    syllablesum=0
    word_cunt=0
    for i in clean_text.split():
        val=syllable_count(i)
        if val:
            syllablesum += val[0]  
        else:
            missing_syllable.append(syllable_error_index)
        word_cunt += 1

    if word_cunt != 0:
        return syllablesum / word_cunt
    else:
        return 'NaN'  
    syllable_error_index += 1
    
def count_pronouns(text):
    #not made exception for US as it is removed by stopword
    pronoun_pattern = r'\b(i|we|my|ours|us)\b'
    pronouns_found = re.findall(pronoun_pattern, text, flags=re.IGNORECASE)
    pronoun_count = len(pronouns_found)
    
    return pronoun_count

def nltstopwordremover(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x=new_text[:]
    new_text.clear()
    return " ".join(x)

def word_count(text):
    unpunctuated=remove_punc(text) 
    readytext=nltstopwordremover(text)
    wordCount=len(readytext.split())
    return wordCount

def avgwordlen(text):
    words = text.split()
    total_characters = sum(len(word) for word in words)
    total_words = len(words)
    if total_words > 0:
        average_word_length = total_characters / total_words
    else:
        average_word_length = 0  
    return average_word_length

Calling the above defined functions

In [10]:

sent=[]
word=[]
dict={}
article_Count =0
complex_wordCount=[]
sent_index=0
avg_sentencelength=[]
fog_index=[]
complex_percent=[]
word_cou=[]
missing_syllable=[]
df['sentence_token'].apply(sentence_count)

#finding discrepancies and handling them
ind = []
for idx, value in enumerate(sent):
    if value == 0:
        ind.append(idx)
        sent[idx] = 1
print("Indices where replacements occurred:", ind)

df['b4Cleanarticle'].apply(readability)

df['ComplexPercent']=complex_percent
df['AvgSentenceLength']=avg_sentencelength
df['Avgwordspersent']=avg_sentencelength
df['Complexwordcount']=complex_wordCount
df['FogIndex']=fog_index

df['syllableperword']=df['b4Cleanarticle'].apply(syllableperword)
df['pronoun_count']=df['b4Cleanarticle'].apply(count_pronouns)

from nltk.corpus import stopwords
nltk.download('stopwords')
df['wordCount']=df['b4Cleanarticle'].apply(word_count)
df['AverageWordLength']=df['b4Cleanarticle'].apply(avgwordlen)

Indices where replacements occurred: [35, 48]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<span style="font-family: 'Times New Roman'; font-size: 24px;">Creating the final Dataframe and Exporting it as Excel</span>


In [11]:
finaldict={
    'URL_ID':df['URL_ID'],
    'URL':df['URL'],
    'POSITIVE SCORE':df['positive_score'],
    'NEGATIVE SCORE':df['negative_score'],
    'POLARITY SCORE':df['polarity_score'],
    'SUBJECTIVITY SCORE':df['subjectivity_score'],
    'AVG SENTENCE LENGTH':df['AvgSentenceLength'],
    'PERCENTAGE OF COMPLEX WORDS':df['ComplexPercent'],
    'FOG INDEX':df['FogIndex'],
    'AVG NUMBER OF WORDS PER SENTENCE':df['AvgSentenceLength'],
    'COMPLEX WORD COUNT':df['Complexwordcount'],
    'WORD COUNT':df['positive_score'],
    'SYLLABLE PER WORD':df['syllableperword'],
    'PERSONAL PRONOUNS':df['pronoun_count'],
    'AVG WORD LENGTH':df['Avgwordspersent']
}
finaldf=pd.DataFrame(finaldict)
print(finaldf)
file_name = 'Output Data Structure.xlsx'
finaldf.to_excel(file_name)

             URL_ID                                                URL  \
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...   
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...   
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...   
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...   
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...   
..              ...                                                ...   
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...   
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...   
97  blackassign0098  https://insights.blackcoffer.com/contribution-...   
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...   
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...   

    POSITIVE SCORE  NEGATIVE SCORE  POLARITY SCORE  SUBJECTIVITY SCORE  \
0               27               6   

In [13]:
import pandas as pd

# Print the version of Pandas
print("Pandas version:", pd.__version__)


Pandas version: 1.5.3
